In [70]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [62]:
driver = webdriver.Safari(executable_path='/usr/bin/safaridriver')

url = "https://platesmania.com/hu/foto18683281"
driver.get(url)


image = driver.find_element_by_xpath("/html/body/div[2]/img")
title = image.get_attribute("title")
print(title)
license_plate = title[:7]
print(license_plate)

image_url = image.get_attribute("src")
print(image_url)

driver.close()


HOX-037, BMW 3 Series 3rd gen (E36), 1990–2000
HOX-037
http://img03.platesmania.com/220318/o/18683281.jpg


In [119]:
def closead(driver):
    #with cross
    try:
        driver.find_element_by_xpath('//*[@id="dismiss-button"]/div/svg/path[1]').click()
        
    except Exception as e:
        if e != "Message: \n":
            print(e)
    
    #with "close" button
    try:
        driver.find_element_by_xpath('//*[@id="dismiss-button"]/div/span').click()
        
    except Exception as e:
        if e != "Message: \n":
            print(e)

In [123]:
#initialize driver
driver = webdriver.Safari(executable_path='/usr/bin/safaridriver')

#open webpage
url = "https://platesmania.com/hu/gallery"
driver.get(url)
time.sleep(1)

#click accept cookies
try:
    driver.find_element_by_xpath('//*[@id="cl-consent"]/div[1]/div[1]/div[3]/a[2]').click()
except:
    print("nocookies")

closead(driver)

#scroll through galleries
for currentgallery in range(2):
    
    time.sleep(2)
    
    for currentimage in range(3):
        
        #close ad
        closead(driver)
        
        #click the thumbnails in the current gallery
        thumbnail_row = 4 + currentimage // 2
        thumbnail_col = 1 + currentimage % 2
        thumbnail_xpath = '/html/body/div[1]/div[6]/div/div[1]/div[' + str(thumbnail_row) + ']/div[' + str(thumbnail_col) + ']/div/div[2]/div[1]/a/img'
        try:
            driver.find_element_by_xpath(thumbnail_xpath).click()
        except:
            closead(driver)
            driver.find_element_by_xpath(thumbnail_xpath).click()
        
        time.sleep(3)
        
        #go back to gallery
        driver.back()
        
        time.sleep(3)
        
        try:
            driver.find_element_by_xpath("/html/body/div[1]/div[6]/div/div[1]/div[3]/ul/li[8]/a").click()
            closead(driver)

        except Exception as e:
            time.sleep(1)
            closead(driver)
            driver.find_element_by_xpath("/html/body/div[1]/div[6]/div/div[1]/div[3]/ul/li[8]/a").click()
        
driver.close()
print("success!")

"""
image = driver.find_element_by_xpath("/html/body/div[2]/img")
title = image.get_attribute("title")
print(title[:7])

image_url = image.get_attribute("src")
print(image_url)

driver.close()
"""

nocookies
Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 

Message: 



WebDriverException: Message: 


In [93]:
'/html/body/div[1]/div[6]/div/div[1]/div[4]/div[1]/div/div[2]/div[1]/a/img'

'/html/body/div[1]/div[6]/div/div[1]/div[4]/div[2]/div/div[2]/div[1]/a/img'

'/html/body/div[1]/div[6]/div/div[1]/div[5]/div[1]/div/div[2]/div[1]/a/img'

'/html/body/div[1]/div[6]/div/div[1]/div[3]/ul/li[8]/a'

2 % 2

0